In [1]:
import pandas as pd
import time
# import selenium
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager 
from selenium.webdriver.common.by import By

from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions

In [147]:
options = webdriver.ChromeOptions()
options.binary_location = "C:\Program Files\Google\Chrome Beta\Application\chrome.exe"
driver = webdriver.Chrome(executable_path=r'C:\Users\a-gue\.wdm\drivers\chromedriver\win32\104.0.5112.20\chromedriver.exe', chrome_options=options)

c:\users\a-gue\onedrive\documentos\python_projects\panama_apt_pricing\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  This is separate from the ipykernel package so we can avoid doing imports until
c:\users\a-gue\onedrive\documentos\python_projects\panama_apt_pricing\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: use options instead of chrome_options
  This is separate from the ipykernel package so we can avoid doing imports until


In [148]:
# page_url = "https://www.encuentra24.com/panama-en/real-estate-for-sale-apartments-condos?q=number.50"
page_url = "https://www.encuentra24.com/panama-en/real-estate-for-sale-apartments-condos/prov-panama?q=withcat.real-estate-for-sale-apartments-condos|number.50"

In [149]:
driver.get(page_url)

__________________

### Start Here | Get link per pages:
Below a function that gets all pages link in encuentra24


In [151]:
def get_links_of_pages(pages_num):
    '''Get the links of the number of pages on encuentra 24 main for panama new apartaments
    - pages_num: number of pages that we will scrapt
    '''
    url_list = []
    for i in range(pages_num):

        # _url_decomposed = "https://www.encuentra24.com/panama-en/real-estate-for-sale-apartments-condos." + str(i+1) + "?q=number.50"
        _url_decomposed = "https://www.encuentra24.com/panama-en/real-estate-for-sale-apartments-condos/prov-panama." + str(i+1) + "?q=withcat.real-estate-for-sale-apartments-condos|number.50"
        url_list.append(_url_decomposed)
    return url_list

In [152]:
url_list = get_links_of_pages(247)
url_list

['https://www.encuentra24.com/panama-en/real-estate-for-sale-apartments-condos/prov-panama.1?q=withcat.real-estate-for-sale-apartments-condos|number.50',
 'https://www.encuentra24.com/panama-en/real-estate-for-sale-apartments-condos/prov-panama.2?q=withcat.real-estate-for-sale-apartments-condos|number.50',
 'https://www.encuentra24.com/panama-en/real-estate-for-sale-apartments-condos/prov-panama.3?q=withcat.real-estate-for-sale-apartments-condos|number.50',
 'https://www.encuentra24.com/panama-en/real-estate-for-sale-apartments-condos/prov-panama.4?q=withcat.real-estate-for-sale-apartments-condos|number.50',
 'https://www.encuentra24.com/panama-en/real-estate-for-sale-apartments-condos/prov-panama.5?q=withcat.real-estate-for-sale-apartments-condos|number.50',
 'https://www.encuentra24.com/panama-en/real-estate-for-sale-apartments-condos/prov-panama.6?q=withcat.real-estate-for-sale-apartments-condos|number.50',
 'https://www.encuentra24.com/panama-en/real-estate-for-sale-apartments-cond

In [11]:
# # get individual url
# url_by_add_test = driver.find_elements(By.XPATH, "//div[@class='ann-box-details']/a")
# # url_by_add[0].get_attribute("href")
# url_by_add_test[0].get_attribute("href")   

### Get Individual Links per Add

In [153]:
def get_url_by_add(driver, url_list):
    '''Gets url list by each individual add
    - return: list of url of each add
    '''
    url_by_add = []
    # iterate over link pages
    for i in url_list:
        # get url on driver
        driver.get(i)
        time.sleep(3)
        # find all box details
        box_details = driver.find_elements(By.XPATH, "//div[@class='ann-box-details']/a")

        for j in box_details:
            url_by_add.append(j.get_attribute("href"))
    return url_by_add

In [154]:
url_by_add = get_url_by_add(driver, url_list)

In [156]:
# save to csv all urls
df_by_add = pd.DataFrame(url_by_add, columns=["url_by_add"])
df_by_add.to_csv("url_by_add.csv")

### Get area per add

In [123]:
def get_area(labels_in_add, values_in_add):
    # eval if key word for price is present
    if 'M²' in labels_in_add:
        indx = labels_in_add.index('M²')
    elif 'M² of construction' in labels_in_add:
        indx = labels_in_add.index('M² of construction')
    elif 'M²:' in labels_in_add:
        indx = labels_in_add.index('M²:')
    elif 'M² of construction:' in labels_in_add:
        indx = labels_in_add.index('M² of construction:')
    else:
        indx = None
        
    # if keyword for area not present use nan
    if indx != None:
        area = values_in_add[indx]
    else:  
         area = 'NaN'
    
    return area
    

### Get Price per add

In [141]:
def get_price(labels_in_add, values_in_add, driver):
    '''Return Price of Apartment if present; NaN otherwise
    - labels_in_add: labels of full description
    - values_in_add: values of full description
    '''

    # eval if key word for price is present
    if 'Price' in labels_in_add:
        indx = labels_in_add.index('Price')
        
    elif 'Precio' in labels_in_add:
        indx = labels_in_add.index('Precio')
        
    elif 'Price:' in labels_in_add:
        indx = labels_in_add.index('Price:')
        
    elif 'Precio:' in labels_in_add:
        indx = labels_in_add.index('Precio:')
    else:
        indx = None
        
    # if keyword for price not present use selenium xpath
    if indx == None:
        try:
            price = driver.find_element(By.XPATH, "//div/div[@class='d3-property-headline__text-element d3-property-headline__text-element--price']").text
        except:
            price = 'NaN'
    else:
        
        price = values_in_add[indx]
    
    return price

### Get Bathrooms per add

In [129]:
def get_bath(labels_in_add, values_in_add):
    
    # eval if key word for bathroom is present
    if 'Bathrooms' in labels_in_add:
        indx = labels_in_add.index('Bathrooms')
        
    elif 'Bathrooms:' in labels_in_add:
        indx = labels_in_add.index('Bathrooms:')

    else:
        indx = None
        
    # if keyword for area not present use nan
    if indx != None:
        bath = values_in_add[indx]
    else:  
         bath = 'NaN'
    
    return bath

### Get Bedrooms per add

In [133]:
def get_bed(labels_in_add, values_in_add):
    
    # eval if key word for bedroom is present
    if 'Bedrooms' in labels_in_add:
        indx = labels_in_add.index('Bedrooms')
        
    elif 'Bedrooms:' in labels_in_add:
        indx = labels_in_add.index('Bedrooms:')

    else:
        indx = None
        
    # if keyword for bedroom not present use nan
    if indx != None:
        bed = values_in_add[indx]
    else:  
         bed = 'NaN'
    
    return bed

### Get Parkings per add

In [134]:
def get_park(labels_in_add, values_in_add):
    
    # eval if key word for parking is present
    if 'Parking' in labels_in_add:
        indx = labels_in_add.index('Parking')
        
    elif 'Parking:' in labels_in_add:
        indx = labels_in_add.index('Parking:')

    else:
        indx = None
        
    # if keyword for parking not present use nan
    if indx != None:
        park = values_in_add[indx]
    else:  
         park = 'NaN'
    
    return park

# Get full description per add

In [157]:
# # Case 1

# # def get_attributes_case_3(driver, add_price, add_loc, add_price_area_labl, add_price_area_val, add_area_labl, add_area_val, add_bed_labl, add_bed_val,
# #                          add_bath_labl, add_bath_val, add_park_labl, add_park_val):
    
# full_description_val = driver.find_elements(By.XPATH, "//dl/dd[@class='d3-property-insight__attribute-value']")
# full_description_label = driver.find_elements(By.XPATH, "//dl/dt[@class='d3-property-insight__attribute-title']")
# descrip_list = [] # len = 5| price, area, rooms, bathrooms, parkings
# rang = len(full_description_val)
# for i in range(rang):
#     print(full_description_label[i].text)
#     # import pdb; pdb.set_trace()
#     print(full_description_val[i].text)


In [77]:
# Case 2
full_description_label =  driver.find_elements(By.XPATH,"//li/span[@class='info-name']")
full_description_val = driver.find_elements(By.XPATH, "//li/span[@class='info-value']")
for j in range(len(full_description_label)):
    print(j, full_description_label[j].text, full_description_val[j].text)
    # 3: price
    # 2: location
    # 4: price area val
    # 14: area
    # 11: Bedrooms
    # 12: Bathroom
    # 13: Parking
    

In [137]:
# Case 3
def get_attributes_case_3(driver, add_price,  
                          add_area_val, add_bed_val,
                          add_bath_val, add_park_val):
    '''
    Get the attributes and append to respective list in case of different formating on the add. This is not efficient method as it adds another for loop.
    - driver: browser object
    - remaining params are list to append the data
    '''
    
    full_desc_labl = driver.find_elements(By.XPATH, "//div[@class='col-800']/ul/li/span[@class='info-name']")
    full_desc_val = driver.find_elements(By.XPATH, "//div[@class='col-800']/ul/li/span[@class='info-value']")
    
    if len(full_desc_labl) == 0:
        print("initial link to all descriptions did not work, Use the alternative full description")
        
        full_desc_labl = driver.find_elements(By.XPATH, "//dl/dt[@class='d3-property-insight__attribute-title']")
        full_desc_val = driver.find_elements(By.XPATH, "//dl/dd[@class='d3-property-insight__attribute-value']")
        
        if len(full_desc_labl) == 0:
            print("find a new way to get attributtes")
            raise NotImplementedError
        
    labels_in_add = []
    values_in_add = []
    for i in range(len(full_desc_labl)):

        # print(i, full_desc_labl[i].text, full_desc_val[i].text, '<< These are the label and value at each index on the description')
        
        labels_in_add.append(full_desc_labl[i].text)
        values_in_add.append(full_desc_val[i].text)
    
    

    # get price
    price_val = get_price(labels_in_add, values_in_add, driver)

    # get area
    area_val = get_area(labels_in_add, values_in_add)
    
    # get bath    
    bath_val = get_bath(labels_in_add, values_in_add)
     
    # get bed
    bed_val = get_bed(labels_in_add, values_in_add)

 
    # get park
    park_val = get_park(labels_in_add, values_in_add)



    add_price.append(price_val)

    add_area_val.append(area_val)

    add_bed_val.append(bed_val)

    add_bath_val.append(bath_val)

    add_park_val.append(park_val)

    

_________________________________________________________

# Data Extraction


In [158]:
df = pd.read_csv("url_by_add.csv")

In [159]:
url_by_add = df['url_by_add']

add_title = []
add_price = []
add_area_val = []
add_bed_val = []
add_bath_val = []
add_park_val = []

# iterate over links
for url in url_by_add[0:]:
    # print(url)
    # get url on driver
    driver.get(url)
    # sleep 10 seconds just in case
    time.sleep(5)
    
    
    # get add and append
    title = driver.find_element(By.XPATH, "//div/div/h1").text
    add_title.append(title)
    print("Title = ", title)

    get_attributes_case_3(driver, add_price,  
                          add_area_val, add_bed_val,
                          add_bath_val, add_park_val)      


Title =  Panama Viejo Residences, en Panama Viejo
initial link to all descriptions did not work, Use the alternative full description
Title =  Armonía en Cinta Costera
initial link to all descriptions did not work, Use the alternative full description
Title =  Coco Place en Coco del Mar, San Francisco
initial link to all descriptions did not work, Use the alternative full description
Title =  Apartments in El Cangrejo | El Cangrejo Porto Bello Downtown Remodeled Apartment
Title =  Apartments in El Bosque | El Bosque Alsacia Towers Semi Furnished Remodeled Apartment
Title =  Apartments in Carrasquilla | Torres de Castilla Carrasquilla Apartment w/ Appliances
Title =  Apartments in Avenida Balboa | VENTA / ALQUILER APARTAMENTO PH COSTANERA BELLA VISTA
Title =  Apartments in San Francisco | Sale of apartment in San Francisco
Title =  Apartments in Punta Pacífica | Oceanfront OASIS ON THE BAY in Punta Pacifica
Title =  Apartments in Coco del Mar | VENTA APARTAMENTO PH QUADRAT SAN FRANCISCO

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div/div/h1"}
  (Session info: chrome=105.0.5195.37)
Stacktrace:
Backtrace:
	Ordinal0 [0x0036AF23+2207523]
	Ordinal0 [0x003030E1+1781985]
	Ordinal0 [0x00213FE8+802792]
	Ordinal0 [0x00242290+991888]
	Ordinal0 [0x0024252B+992555]
	Ordinal0 [0x0026F6E2+1177314]
	Ordinal0 [0x0025D634+1103412]
	Ordinal0 [0x0026D9F2+1169906]
	Ordinal0 [0x0025D406+1102854]
	Ordinal0 [0x002375F0+947696]
	Ordinal0 [0x002384E6+951526]
	GetHandleVerifier [0x00612A12+2733954]
	GetHandleVerifier [0x00604E28+2677656]
	GetHandleVerifier [0x003F46EA+513114]
	GetHandleVerifier [0x003F3796+509190]
	Ordinal0 [0x00309CEB+1809643]
	Ordinal0 [0x0030E448+1827912]
	Ordinal0 [0x0030E535+1828149]
	Ordinal0 [0x00317960+1866080]
	BaseThreadInitThunk [0x751EFA29+25]
	RtlGetAppContainerNamedObjectPath [0x77047A9E+286]
	RtlGetAppContainerNamedObjectPath [0x77047A6E+238]


# Backup data

In [160]:
df_bckp = pd.DataFrame({"add_title":add_title, 
                   "add_price": add_price, 
                   "add_area_val": add_area_val,
                   "add_bed_val": add_bed_val,
                   "add_bath_val": add_bath_val,
                   "add_park_val": add_park_val
                  })

In [161]:
df_bckp

,add_title,add_price,add_area_val,add_bed_val,add_bath_val,add_park_val
0,"Panama Viejo Residences, en Panama Viejo","B/.139,000.00",58,2,2,1
1,Armonía en Cinta Costera,"B/.124,500.00",45.24,1,1,NaN
2,"Coco Place en Coco del Mar, San Francisco","B/.233,259.60",79.34,2,2,1
3,Apartments in El Cangrejo | El Cangrejo Porto ...,"B/.145,000.00",92,2,2,1
4,Apartments in El Bosque | El Bosque Alsacia To...,"B/.118,000.00",66,2,2,1
...,...,...,...,...,...,...
711,Apartments in Marbella | Torres Ebbelle. 167mt...,"B/.250,000.00",167,3,2.5,2
712,Apartments in El Dorado | Park View. 123mts 3r...,"B/.169,000.00",123,3,3,2
713,Apartments in Coco del Mar | PH Vision Tower. ...,"B/.570,000.00",280,3,3.5,2
714,Apartments in San Francisco | Keops Tower en v...,"B/.380,000.00",200,3,3.5,2


In [162]:
df_bckp.to_csv("last_data_set.csv")